In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense,Dropout,Conv2D,MaxPooling2D,Flatten
from keras.optimizers import RMSprop

Using TensorFlow backend.


In [2]:
(mnist_train_images,mnist_train_labels),(mnist_test_images,mnist_test_labels) = mnist.load_data()

In [3]:
mnist_train_images.shape[0]

60000

In [4]:
from tensorflow.keras import backend as K

if K.image_data_format() == 'channels_first':
    train_images = mnist_train_images.reshape(mnist_train_images.shape[0], 1, 28, 28)
    test_images = mnist_test_images.reshape(mnist_test_images.shape[0], 1, 28, 28)
    input_shape = (1, 28, 28)
else:
    train_images = mnist_train_images.reshape(mnist_train_images.shape[0], 28, 28, 1)
    test_images = mnist_test_images.reshape(mnist_test_images.shape[0], 28, 28, 1)
    input_shape = (28, 28, 1)
    
train_images = train_images.astype('float32')
test_images = test_images.astype('float32')
train_images /= 255
test_images /= 255

In [5]:
import tensorflow
train_labels = tensorflow.keras.utils.to_categorical(mnist_train_labels, 10)
test_labels = tensorflow.keras.utils.to_categorical(mnist_test_labels, 10)

In [6]:
import matplotlib.pyplot as plt

def display_sample(num):
    #Print the one-hot array of this sample's label 
    print(train_labels[num])  
    #Print the label converted back to a number
    label = train_labels[num].argmax(axis=0)
    #Reshape the 768 values to a 28x28 image
    image = train_images[num].reshape([28,28])
    plt.title('Sample: %d  Label: %d' % (num, label))
    plt.imshow(image, cmap=plt.get_cmap('gray_r'))
    plt.show()
    
display_sample(1234)

[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]


<Figure size 640x480 with 1 Axes>

In [7]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
# 64 3x3 kernels
model.add(Conv2D(64, (3, 3), activation='relu'))
# Reduce by taking the max of each 2x2 block
model.add(MaxPooling2D(pool_size=(2, 2)))
# Dropout to avoid overfitting
model.add(Dropout(0.25))
# Flatten the results to one dimension for passing into our final layer
model.add(Flatten())
# A hidden layer to learn with
model.add(Dense(128, activation='relu'))
# Another dropout
model.add(Dropout(0.5))
# Final categorization from 0-9 with softmax
model.add(Dense(10, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [9]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [10]:
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=10,
                    verbose=2,
                    validation_data=(test_images, test_labels))

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
 - 1263s - loss: 0.1869 - acc: 0.9436 - val_loss: 0.0432 - val_acc: 0.9862
Epoch 2/10
 - 1108s - loss: 0.0761 - acc: 0.9771 - val_loss: 0.0362 - val_acc: 0.9880
Epoch 3/10
 - 1874s - loss: 0.0577 - acc: 0.9825 - val_loss: 0.0304 - val_acc: 0.9905
Epoch 4/10
 - 1997s - loss: 0.0487 - acc: 0.9851 - val_loss: 0.0297 - val_acc: 0.9900
Epoch 5/10
 - 964s - loss: 0.0410 - acc: 0.9871 - val_loss: 0.0283 - val_acc: 0.9917
Epoch 6/10
 - 1442s - loss: 0.0352 - acc: 0.9888 - val_loss: 0.0295 - val_acc: 0.9924
Epoch 7/10
 - 1678s - loss: 0.0330 - acc: 0.9895 - val_loss: 0.0280 - val_acc: 0.9915
Epoch 8/10
 - 4695s - loss: 0.0278 - acc: 0.9911 - val_loss: 0.0295 - val_acc: 0.9911
Epoch 9/10
 - 4380s - loss: 0.0263 - acc: 0.9920 - val_loss: 0.0288 - val_acc: 0.9918
Epoch 10/10
 - 2134s - loss: 0.0253 - acc: 0.9920 - val_loss: 0.0287 - val_acc: 0.9926


In [13]:
score = model.evaluate(test_images, test_labels, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.028747187863102364
Test accuracy: 0.9926
